# Spectral line imaging demonstration

Create a spectral line data set, add a grid components, multiply by the low BEAM, transform exactly to visibilities.

We remove the continuum in the image plane and in the visibility plane, using a second order term in frequency.

We will make use to graphs to speed the processing. These are based on dask.delayed.

In [ ]:
%matplotlib inline

import os
import sys
sys.path.append(os.path.join('..','..'))

results_dir = './results'
os.makedirs(results_dir, exist_ok=True)

from matplotlib import pylab

pylab.rcParams['figure.figsize'] = (12.0, 12.0)
pylab.rcParams['image.cmap'] = 'rainbow'

import numpy

from scipy.interpolate import interp1d

from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.wcs.utils import pixel_to_skycoord

from matplotlib import pyplot as plt

from arl.data.polarisation import PolarisationFrame
from arl.data.parameters import get_parameter
from arl.data.data_models import Visibility
from arl.visibility.base import create_visibility, create_blockvisibility
from arl.visibility.operations import remove_continuum_blockvisibility
from arl.visibility.coalesce import convert_blockvisibility_to_visibility
from arl.skycomponent.operations import create_skycomponent
from arl.skycomponent.operations import create_skycomponent, apply_beam_to_skycomponent

from arl.image.deconvolution import deconvolve_cube, restore_cube
from arl.image.operations import show_image, export_image_to_fits, qa_image, remove_continuum_image
from arl.image.iterators import  image_raster_iter
from arl.visibility.iterators import vis_timeslice_iter
from arl.util.testing_support import create_named_configuration, create_low_test_beam
from arl.imaging import predict_2d, invert_timeslice, create_image_from_visibility, \
    predict_skycomponent_visibility

import logging

log = logging.getLogger()
log.setLevel(logging.INFO)
log.addHandler(logging.StreamHandler(sys.stdout))

We create the visibility. 

In [ ]:
lowcore = create_named_configuration('LOWBD2-CORE')
times = numpy.linspace(-3,+3,3) * (numpy.pi / 12.0)
vnchan=33
frequency = numpy.linspace(1e8, 1.5e8, vnchan)
fitting_mask = numpy.ones([vnchan], dtype='bool')
fitting_mask[vnchan//4:3*vnchan//4] = False
print(fitting_mask)

channel_bandwidth = numpy.array(vnchan*[(frequency[1]-frequency[0])])
phasecentre = SkyCoord(ra=+15.0 * u.deg, dec=-45.0 * u.deg, frame='icrs', equinox='J2000')
vt = create_blockvisibility(lowcore, times, frequency, channel_bandwidth=channel_bandwidth,
                       weight=1.0, phasecentre=phasecentre, polarisation_frame=PolarisationFrame("stokesI"))

Create a grid of components

In [ ]:
npixel = 256
cellsize=0.001
flux = numpy.array(vnchan*[[100.0]])
facets = 4
model = create_image_from_visibility(vt, npixel=npixel, cellsize=cellsize, npol=1, frequency=frequency,
                                    polarisation_frame=PolarisationFrame("stokesI"))
spacing_pixels = npixel // facets
log.info('Spacing in pixels = %s' % spacing_pixels)
spacing = 180.0 * cellsize * spacing_pixels / numpy.pi
centers = -1.5, -0.5, +0.5, +1.5
comps = list()
for iy in centers:
    for ix in centers:
        pra =  int(round(npixel // 2 + ix * spacing_pixels - 1))
        pdec = int(round(npixel // 2 + iy * spacing_pixels - 1))
        sc = pixel_to_skycoord(pra, pdec, model.wcs)
        log.info("Component at (%f, %f) %s" % (pra, pdec, str(sc)))
        comps.append(create_skycomponent(flux=flux, frequency=frequency, direction=sc, 
                                         polarisation_frame=PolarisationFrame("stokesI")))

continuum_model = create_image_from_visibility(vt, npixel=npixel, cellsize=cellsize, npol=1, 
                                               frequency=[frequency[vnchan//2]], nchan=1,
                                               polarisation_frame=PolarisationFrame("stokesI"))

Apply LOW Beam to components and predict the visibilities.

In [ ]:
bm = create_low_test_beam(model=model)
sc = apply_beam_to_skycomponent(comps, bm)
vt = predict_skycomponent_visibility(vt, sc=comps)

In [ ]:
dirty_imlin, sumwt = invert_timeslice(vt, model)

Fit and remove the continuum in the dirty image. This is usually called IMLIN.

In [ ]:
dirty_imlin = remove_continuum_image(dirty_imlin, degree=2, mask=fitting_mask)
print(qa_image(dirty_imlin))

In [ ]:
show_image(dirty_imlin, chan=0, title='IMLIN: Channel 0')
plt.show()
show_image(dirty_imlin, chan=16, title='IMLIN: Channel 16')
plt.show()
show_image(dirty_imlin, chan=32, title='IMLIN: Channel 32')
plt.show()

export_image_to_fits(dirty_imlin, '%s/imaging-spectral-imlin.fits' % (results_dir))

Fit and remove the continuum in the visibility. This is usually called UVLIN.

In [ ]:
vt=remove_continuum_blockvisibility(vt, degree=5, mask=fitting_mask)

In [ ]:
dirty_uvlin, sumwt=invert_timeslice(vt, model)
print(qa_image(dirty_uvlin))

In [ ]:
show_image(dirty_uvlin, chan=0, title='UVLIN: Channel 0')
plt.show()
show_image(dirty_uvlin, chan=16, title='UVLIN: Channel 16')
plt.show()
show_image(dirty_uvlin, chan=32, title='UVLIN: Channel 32')
plt.show()

export_image_to_fits(dirty_uvlin, '%s/imaging-spectral-uvlin.fits' % (results_dir))